In [5]:
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
import statsmodels.formula.api as smf

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [6]:
%pylab inline
%config InlineBackend.figure_formats = ['retina']

sns.set_style("whitegrid", {"font.family": ["serif"]})
sns.set_context("talk")

Populating the interactive namespace from numpy and matplotlib


In [221]:
with open('wine_data_longer.pkl', 'rb') as picklefile: 
    wine_data = pickle.load(picklefile)

In [222]:
wine_data.head()

,Average Rating,Winery,Wine Name,Vintage,Type,Region,Country,Number of Reviews,Average Price
0,4.298098,Veuve Clicquot,Brut (Carte Jaune) Champagne,N.V.,Sparkling wine,Champagne,France,98098,49.99
1,4.187012,Moët & Chandon,Impérial Brut Champagne,N.V.,Sparkling wine,Champagne,France,87012,error
2,3.767924,Casal Garcia,Vinho Verde Branco,N.V.,White wine,Vinho Verde,Portugal,67924,error
3,4.352855,Farnese,Edizione Cinque Autoctoni,N.V.,Red wine,Abruzzo,Italy,52855,error
4,3.847411,Piccini,Memoro Rosso,N.V.,Red wine,Toscana,Italy,47411,10.99


In [223]:
#remove rows with no average price
wine_data = wine_data[wine_data['Average Price'] != 'error']

In [168]:
#move number of reviews out of average rating
'''
def strip_paren(x):
    if "(" in x:
        return x.split("(")[1]
    else:
        pass
'''

In [169]:
'''
wine_data_clean['Number of Reviews'] = wine_data_clean['Average Rating'].apply(strip_paren)
wine_data_clean['Number of Reviews'].replace(')', '')
wine_data_clean['Average Rating'] = wine_data_clean['Average Rating'].apply(lambda x: str(x).split('(')[0])
wine_data_clean['Number of Reviews'] = wine_data_clean['Number of Reviews'].apply(lambda x: str(x).replace(')', ''))
'''

<ipython-input-169-4b8022bec6c3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine_data_clean['Number of Reviews'] = wine_data_clean['Average Rating'].apply(strip_paren)
<ipython-input-169-4b8022bec6c3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine_data_clean['Average Rating'] = wine_data_clean['Average Rating'].apply(lambda x: str(x).split('(')[0])
<ipython-input-169-4b8022bec6c3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [262]:
#get rid of rows with messed up avg rating - its not many
#wine_data = wine_data[~wine_data["Average Rating"].isin(['('])]
wine_data = wine_data[:1400]

In [269]:
#drop rows with no year
wine_data = wine_data[wine_data['Vintage'] != 'N.V.']

In [270]:
#convert to numerical
wine_data['Average Rating'] = wine_data['Average Rating'].astype(float)
wine_data['Number of Reviews'] = wine_data['Number of Reviews'].astype(int)
wine_data['Average Price'] = wine_data['Average Price'].astype(float)
wine_data['Vintage'] = wine_data['Vintage'].astype(int)

<ipython-input-270-fd1063de43cc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine_data['Average Rating'] = wine_data['Average Rating'].astype(float)
<ipython-input-270-fd1063de43cc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine_data['Number of Reviews'] = wine_data['Number of Reviews'].astype(int)
<ipython-input-270-fd1063de43cc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [273]:
wine_data.shape

(1288, 9)

In [274]:
#other Cleaning?

In [ ]:
#FOLLOW PRE-processing notebook
#convert categoricals and

In [ ]:
#convert year into age

In [ ]:
#reg test

LinearRegression()

In [62]:
linreg_model.coef_

array([0.0030779])

In [63]:
linreg_model.intercept_

3.8318511348027555

In [68]:
linreg_model.predict([[20]])

array([3.89340905])

In [69]:
linreg_model.predict([[100]])

array([4.13964074])

In [1]:
wine_data_clean_smaller.describe()

NameError: name 'wine_data_clean_smaller' is not defined